# 주식 데이터 웹 크롤링

**과정**

1. 주식사이트 접속해서 필요한 영역 검색
2. 해당 영역의 태그 및 속성 파악
3. 데이터 추출 및 전처리
4. MySQL로 DB에 테이블 형태로 저장
5. DB에서 데이터 불러와서 정보 가공 및 시각화

### 주식 사이트에 접속

http://comp.fnguide.com

# 주식 데이터 크롤링

### 기존 정보 웹 크롤링

In [1]:
# 삼전 정보 불러오기

from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN="
html = urlopen(url)
bs_obj = BeautifulSoup(html, "html.parser")
print(bs_obj)


<!DOCTYPE html>

<html lang="ko">
<head>
<link href="https://cdn.fnguide.com/SVO2/css/compeach.css?ver2109131021" rel="stylesheet"/>
<!--[if lte IE 9]><link rel="stylesheet" href="https://cdn.fnguide.com/SVO2/css/ie9.css?ver3"><![endif]-->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<!-- <meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0" name="viewport"> -->
<script src="https://cdn.fnguide.com/SVO2/js/lib/jquery-1.11.1.min.js?v=1.0.6" type="text/javascript"></script>
<script src="https://cdn.fnguide.com/SVO2/js/comp_load.js?v=1.0.6" type="text/javascript"></script>
<script src="https://cdn.fnguide.com/SVO2/js/lib/jquery.autocomplete.js?v=1.0.6" type="text/javascript"></script>
<link href="https://cdn.fnguide.com/SVO2/css/jquery.autocomplete.css?v=1.0.1" rel="stylesheet" type="text/css">
<link href="https://cdn.fnguide.com/SVO2/img/CompanyGuide.ico" rel="SHORTC

날짜 추출

In [2]:
date1 = bs_obj.find("span", {"class":"date"})
print(date1.text)

[2022/03/04]


In [3]:
# 원하는 날짜 형식으로 바꾸자

date2 = date1.text
date = date2.replace('[','').replace(']','').replace('/','-')
print(date)

2022-03-04


종목 이름

In [4]:
corp_name1 = bs_obj.find_all("h1", {"id":"giName"})
print(corp_name1)

[<h1 id="giName">삼성전자</h1>]


In [5]:
corp_name = corp_name1[0].text
print(corp_name)

삼성전자


종목 코드

In [6]:
code1 = bs_obj.find_all("div", {"class":"corp_group1"})
print(code1)

[<div class="corp_group1">
<h1 id="giName">삼성전자</h1>
<h2>005930</h2>
<span class="etc1">|</span>
<!--
					<h3>SamsungElec</h3>
					<span class="etc1">&#124;</span>
					-->
<a class="btn_shome" href="javascript:goHompage('http://www.samsung.com')" id="btn_shome"><span>홈페이지</span></a>
<dl id="dl_btn_shome" style="display:none;"><dt>홈페이지</dt><dd id="dd_btn_shome">http://www.samsung.com</dd></dl>
<a class="btn_sphone" href="javascript:void(0)" id="btn_sphone"><span>전화번호</span></a>
<dl id="dl_btn_sphone" style="display:none;"><dt>전화번호</dt><dd id="dd_btn_sphone">031-200-1114 | IR 담당자 02-2255-9000</dd></dl>
<a class="btn_spmap" href="javascript:void(0)" id="btn_spmap"><span>주소</span></a>
<dl id="dl_btn_spmap" style="display:none;"><dt>주소</dt><dd id="dd_btn_spmap">경기도 수원시 영통구 삼성로 129 (매탄동)</dd></dl>
<p class="stxt_group">
<span class="stxt stxt1">KSE  코스피 전기,전자</span>
<span id="strMarketTxt" style="display:none">코스피 전기,전자</span>
<span class="etc2">|</span>
<span class="stxt stxt2">FICS  휴대폰

In [8]:
code2 = code1[0].find("h2")   # div태그 안의 h2태그 안에 있음
print(code2)

<h2>005930</h2>


In [9]:
code = code2.text
print(code)

005930


주가 정보

In [10]:
stock_price1 = bs_obj.find("span", {"id":"svdMainChartTxt11"})
print(stock_price1)

<span id="svdMainChartTxt11">71,500</span>


In [11]:
# 온전히 숫자만 추출

stock_price2 = stock_price1.text
stock_price = int(stock_price2.replace(",", "").strip())
print(stock_price)

71500


외국인 보유 비중

In [12]:
fgn_own_ratio1 = bs_obj.find("span", {"id":"svdMainChartTxt12"})
print(fgn_own_ratio1)

<span id="svdMainChartTxt12">52.05</span>


In [13]:
fgn_own_ratio = float(fgn_own_ratio1.text)
print(fgn_own_ratio)

52.05


상대 수익률

In [14]:
rel_return1 = bs_obj.find("span", {"id":"svdMainChartTxt13"})
print(rel_return1)

<span id="svdMainChartTxt13">-13.23</span>


In [15]:
rel_return = float(rel_return1.text)
print(rel_return)

-13.23


### 상단 테이블 웹 크롤링

In [16]:
up_list = bs_obj.find("div", {"class":"corp_group2"})
print(up_list)

<div class="corp_group2" id="corp_group2">
<dl>
<dt>
<dl style="display:none;"><dt>PER(Price Earning Ratio)</dt><dd>전일자 보통주 수정주가 / 최근 결산 EPS(주당순이익) <br/>* EPS = 당기순이익 / 수정평균발행주식수<br/>* 최근결산은 2021/12 (연간) 기준임.</dd></dl>
<a class="tip_in" href="javascript:void(0)" id="h_per">PER</a>
</dt>
<dd>12.38</dd>
</dl>
<dl>
<dt>
<dl style="display:none;"><dt>12M PER</dt><dd>전일자 보통주 수정주가 / 12개월 Forward EPS</dd></dl>
<a class="tip_in" href="javascript:void(0)" id="h_12m">12M PER</a>
</dt>
<dd>10.52</dd>
</dl>
<dl>
<dt>
<dl style="display:none;"><dt>업종 PER</dt><dd>시장대표업종||SUM(구성종목 시가총액)/SUM(구성종목 당기순이익)<br/>* 전일자 보통주 시가총액 기준<br/>* 당기순이익은  최근결산 2021/12 (연간) 기준임.</dd></dl>
<a class="tip_in" href="javascript:void(0)" id="h_u_per"><span class="ko">업종</span> PER</a>
</dt>
<dd>21.53</dd>
</dl>
<dl>
<dt>
<dl style="display:none;"><dt>PBR(Price Book-value Ratio)</dt><dd>전일자 보통주 수정주가 / 최근 결산기 BPS(주당순자산) <br/>* BPS=(지배주주지분-자기주식) / 무상조정기말주식수(우선주 및 자사주 포함) <br/>* 최근결산은 2021/12 (연간) 기준임.</dd></dl>
<a class="tip_in

In [17]:
# 용어를 나타내는 dd 태그 안의 내용을 추출한다
dd = up_list.find_all("dd")
print(dd)

[<dd>전일자 보통주 수정주가 / 최근 결산 EPS(주당순이익) <br/>* EPS = 당기순이익 / 수정평균발행주식수<br/>* 최근결산은 2021/12 (연간) 기준임.</dd>, <dd>12.38</dd>, <dd>전일자 보통주 수정주가 / 12개월 Forward EPS</dd>, <dd>10.52</dd>, <dd>시장대표업종||SUM(구성종목 시가총액)/SUM(구성종목 당기순이익)<br/>* 전일자 보통주 시가총액 기준<br/>* 당기순이익은  최근결산 2021/12 (연간) 기준임.</dd>, <dd>21.53</dd>, <dd>전일자 보통주 수정주가 / 최근 결산기 BPS(주당순자산) <br/>* BPS=(지배주주지분-자기주식) / 무상조정기말주식수(우선주 및 자사주 포함) <br/>* 최근결산은 2021/12 (연간) 기준임.</dd>, <dd>1.64</dd>, <dd>{최근 결산기 보통주 DPS(현금, 무상조정) / 전일자 보통주 수정주가} *100<br/>* 최근결산은 2021/12 (연간) 기준임.</dd>, <dd>2.02%</dd>]


In [21]:
# PER : Price Earning Ratio

per = float(dd[1].text)
print(per)

12.38


In [22]:
# 12M PER : 12월 뒤의 예상 PER

per_12m = float(dd[3].text)
print(per_12m)

10.52


In [23]:
# 업종 PER : 해당 업종의 PER

per_ind = float(dd[5].text)
print(per_ind)

21.53


In [24]:
# PBR : Pricce to Book Ratio (주가순자산비율)

pbr = float(dd[7].text)
print(pbr)

1.64


In [28]:
# 배당 수익률

div_yid1 = dd[9].text
div_yid = float(div_yid1.replace('%', ''))
print(div_yid)

2.02


### 시세현황 테이블 웹 크롤링

In [30]:
# 테이블의 위치를 찾고, td 태그 속에 있는 정보를 다 크롤링 한다

table1 = bs_obj.find("div", {"id":"div1"})
table2 = table1.find_all("td")
print(table2)

[<td class="r">71,500/ <span class="tcb">-1,400</span></td>, <td class="cle r"> 13,409,634</td>, <td class="r">86,000/ 68,800</td>, <td class="cle r"> 9,606</td>, <td class="r"><span class="tcb">-3.38</span>/ <span class="tcb">-5.42</span>/ <span class="tcb">-6.66</span>/ <span class="tcb">-13.23</span>
<input id="c3M" type="hidden" value="-5.42"/><input id="c1Y" type="hidden" value="-13.23"/><input id="c3Y" type="hidden" value="59.42"/></td>, <td class="cle r">52.05</td>, <td class="r">4,807,385</td>, <td class="cle r">0.96367</td>, <td class="r">4,268,395</td>, <td class="cle r">100</td>, <td class="r">5,969,782,550/ 822,886,700</td>, <td class="cle r">4,461,378,718 / 74.73</td>]


In [31]:
# 거래량

volume1 = table2[1].text
volume = int(volume1.replace(',','').strip())
print(volume)

13409634


In [33]:
# 거래대금

trans_price1 = table2[3].text
trans_price = int(trans_price1.replace(',','').strip())
print(trans_price)

9606


In [34]:
# 시가총액 (우선주 포함)

mk_cpt_pfr1 = table2[6].text
mk_cpt_pfr = int(mk_cpt_pfr1.replace(',','').strip())
print(mk_cpt_pfr)

4807385


In [35]:
# 시가총액 (보통주)

mk_cpt_cm1 = table2[8].text
mk_cpt_cm = int(mk_cpt_cm1.replace(',','').strip())
print(mk_cpt_cm)

4268395


**결과 모음**

In [36]:
res = [date, corp_name, code, stock_price, fgn_own_ratio, rel_return, per,
      per_12m, per_ind, pbr, div_yid, volume, trans_price, mk_cpt_pfr, mk_cpt_cm]
print(res)

['2022-03-04', '삼성전자', '005930', 71500, 52.05, -13.23, 12.38, 10.52, 21.53, 1.64, 2.02, 13409634, 9606, 4807385, 4268395]


# MySQL 기초

데이터의 양이 적을 때는 엑셀로 충분..

-> 대용량에 대해서는 DB를 이용하는 것이 효율적

### 데이터베이스와 MySQL

DBMS : DB를 관리하는 프로그램 ex) 오라클, MySQL, MongoDB

### MySQL 다운로드

https://mysql.com